# Adeptus Optimus Proto

# Calculus

In [8]:
import scipy.special
import numpy as np
import random
import re
import json

In [102]:
# Utils
def map_7_to_None(v):
    return None if v == 7 else v

class RequirementFailError(Exception):
    pass

def require(predicate, error_message):
    if not(predicate):
        raise RequirementFailError(error_message)

require(True, "bla")

thrown_message = ""
try:
    require(False, "bla")
except RequirementFailError as e:
    thrown_message = str(e)
assert(thrown_message == "bla")
    
    
    
def roll_D6():
    return random.randint(1, 6)
assert(len({roll_D6() for _ in range(1000)}) == 6)

def roll_D3():
    return (random.randint(1, 6) + 1) // 2
assert(len({roll_D3() for _ in range(1000)}) == 3)

class DiceExpr:
    def __init__(self, n, dices_type=None):
        self.n = n
        self.dices_type = dices_type
        
        if self.dices_type is None:
            self.avg = n
        else:
            self.avg = n * (self.dices_type + 1)/2
    
    def roll(self):
        if self.dices_type is None:
            return self.n
        else:
            if self.dices_type == 3:
                return sum([roll_D3() for _ in range(self.n)])
            elif self.dices_type == 6:
                return sum([roll_D6() for _ in range(self.n)])
            else:
                raise AttributeError(f"Unsupported dices_type D{self.dices_type}")
    
        
    def __str__(self):
        if self.dices_type is None:
            return str(self.n)
        else:
            return f"{self.n if self.n > 1 else ''}D{self.dices_type}"

        
assert(str(DiceExpr(5, 3)) == "5D3")
assert(str(DiceExpr(1, 6)) == "D6")
assert(str(DiceExpr(10, None)) == "10")

dice_5D3 = DiceExpr(5, 3)
assert(len({dice_5D3.roll() for _ in range(10000)}) == 5*3 -5 + 1)
dice_4D6 = DiceExpr(4, 6)
assert(len({dice_4D6.roll() for _ in range(10000)}) == 4*6 -4 + 1)          
    
def parse_dice_expr(d, complexity_threshold=36):
    assert(type(d) is str)
    groups = re.fullmatch(r"([1-9][0-9]*)?D([36])?|([0-9]+)", d)
    res = None
    try:
        if groups.group(1) is None:
            n_dices = 1
        else:
            n_dices = int(groups.group(1))
        dices_type = int(groups.group(2))
        if(n_dices*(1 if dices_type is None else dices_type) > complexity_threshold):
            res = None
        else:
            res = DiceExpr(n_dices, dices_type)
    except TypeError:
        try:
            flat = int(groups.group(3))
            res = DiceExpr(flat)
        except TypeError:
            res = None
    finally:
        return res
    
assert(parse_dice_expr("4D3").avg == 8)
assert(parse_dice_expr("5").avg == 5)
assert(parse_dice_expr("D7") is None)
assert(parse_dice_expr("0D6") is None)
assert(parse_dice_expr("0").avg == 0)
assert(parse_dice_expr("7D6") is None)
assert(parse_dice_expr("D3").avg == 2)
assert(parse_dice_expr("3D3").avg == 6)
assert(parse_dice_expr("D6").avg == 3.5)

def parse_roll(roll):
    res = re.fullmatch(r"([23456])\+", roll)
    if res is None:
        return None
    else:
        return int(res.group(1))

assert(parse_roll("1+") is None)
assert(parse_roll("1+") is None)
assert(parse_roll("2+") == 2)
assert(parse_roll("3+") == 3)
assert(parse_roll("6+") == 6)
assert(parse_roll("7+") is None)
assert(parse_roll("3") is None)

def float_eq(a, b):
    return np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)

def prob_by_roll_result(dice_expr):
    if dice_expr.dices_type is None:
        return {dice_expr.n: 1}
    else:
        roll_results_counts = {}
        def f(n, current_sum):
            if n == 0:
                roll_results_counts[current_sum] = roll_results_counts.get(current_sum, 0) + 1
            else:
                for i in range(1, dice_expr.dices_type + 1):
                    f(n - 1, current_sum + i)
        f(dice_expr.n, 0)
        n_cases = sum(roll_results_counts.values())
        for key in roll_results_counts.keys():
            roll_results_counts[key] /= n_cases
        return roll_results_counts
    
assert(prob_by_roll_result(parse_dice_expr("D3")) == {1: 1/3, 2: 1/3, 3: 1/3})
assert(prob_by_roll_result(parse_dice_expr("7")) == {7: 1})
assert(float_eq(1, sum(prob_by_roll_result(parse_dice_expr("2D6")).values())))
assert(prob_by_roll_result(parse_dice_expr("2D6")) == {2: 1/36, 3: 2/36, 4: 3/36, 5: 4/36, 6: 5/36, 7: 6/36, 8: 5/36, 9: 4/36, 10: 3/36, 11: 2/36, 12: 1/36})

In [337]:
# Core Classes
class Bonuses:
    def __init__(self, to_hit, to_wound, props = None):
        assert(to_hit in {-1, 0, 1})
        assert(to_wound in {-1, 0, 1})

        self.to_hit = to_hit
        self.to_wound = to_wound
        self.props = props if props is not None else {}
        
    @classmethod
    def empty(cls):
        return Bonuses(0, 0)
    
Bonuses.empty().to_hit = 0
        
class Weapon:
    def __init__(self, hit, a, s, ap, d, bonuses, points=1):
        self.hit      = parse_dice_expr(hit)
        assert(self.hit is not None)
        self.a        = parse_dice_expr(a)
        assert(self.a is not None)
        self.s        = parse_dice_expr(s)
        assert(self.s is not None)
        self.ap       = parse_dice_expr(ap)
        assert(self.ap is not None)
        self.d        = parse_dice_expr(d)
        assert(self.d is not None)

        self.bonuses  = bonuses   
        assert(type(points) is int and points > 0)
        self.points   = points
        
        
Weapon(hit="5", a="2", s = "4D3", ap = "1", d="D3", bonuses=Bonuses.empty())

        
class Target:
    def __init__(self, t, sv, invu=None, fnp=None, w=1):

        
        assert(invu is None or (type(invu) is int and invu > 0 and invu <= 6))
        self.invu   = invu
        
        assert(fnp is None or (type(fnp) is int and fnp > 0 and fnp <= 6))
        self.fnp   = fnp
        
        assert(type(t) is int and t > 0)
        self.t    = t
        
        assert(type(sv) is int and sv > 0 and sv <= 6)
        self.sv   = sv
        
        assert(type(w) is int and w > 0)
        self.w   = w


Target(8, 3, 5, 6)
Target(8, 3)

In [229]:
# Engine v1

def compute_successes_ratio(modified_necessary_roll, auto_success_on_6=True):
    necessary_roll = modified_necessary_roll
    if modified_necessary_roll <= 1:
        necessary_roll = 2  # roll of 1 always fails
    if modified_necessary_roll >= 7:
        if auto_success_on_6:
            necessary_roll = 6  # roll of 6 always succeeds
        else:
            return 0
    return (7 - necessary_roll)/6

def compute_n_hits_ratio(weapon):
    return compute_successes_ratio(weapon.hit.avg - weapon.bonuses.to_hit)


assert(compute_n_hits_ratio(Weapon(hit="2", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses.empty())) == 5/6)
assert(compute_n_hits_ratio(Weapon(hit="5", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses.empty())) == 1/3)
assert(compute_n_hits_ratio(Weapon(hit="3", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses.empty())) == 2/3)
assert(compute_n_hits_ratio(Weapon(hit="3", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(-1, 0))) == 1/2)
assert(compute_n_hits_ratio(Weapon(hit="2", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(+1, 0))) == 5/6)  # roll of 1 always fails
assert(compute_n_hits_ratio(Weapon(hit="6", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(-1, 0))) == 1/6)  # roll of 6 always succeeds

def compute_necessary_wound_roll(f, e):
    if f >= 2*e:
        return 2
    elif f > e:
        return 3
    elif f == e:
        return 4
    elif f <= e/2:
        return 6
    else:
        assert(f < e)
        return 5

def compute_n_wounds_ratios(weapon, target):
    return [prob*compute_successes_ratio(compute_necessary_wound_roll(f, target.t) - weapon.bonuses.to_wound)
     for f, prob in prob_by_roll_result(weapon.s).items()]

def compute_n_wounds_ratio(weapon, target):
    return sum(compute_n_wounds_ratios(weapon, target))

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=4, sv=3)
) == 1/2)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=8, sv=3)
) == 1/6)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, -1)),
    Target(t=8, sv=3)
) == 1/6)  # roll of 6 always succeeds

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=2, sv=3)
) == 5/6)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, +1)),
    Target(t=2, sv=3)
) == 5/6)  # roll of 1 always fails

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=12, sv=3)
) == 1/6)

assert(float_eq(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=1, sv=3)
), 5/6))

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=4, sv=3)
)< 1/2)



def n_figs_slained(weapon, target):
    hit_ratio = compute_n_hits_ratio(weapon)
    wound_ratio = compute_n_wounds_ratio(weapon, target)
    return weapon.a.avg * hit_ratio * wound_ratio

def total_n_figs_slained(weapons_list, target):
    print("tatata")

In [230]:
n_figs_slained(
    Weapon(hit="5", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=8, sv=3)
)

1.0

In [339]:
# Engine v2
def dispatch_density_key(previous_density_key, next_density_prob):
    assert(type(previous_density_key) is int)
    assert(previous_density_key >= 0)
    assert(0 < next_density_prob and next_density_prob <= 1)
    n = previous_density_key
    p= next_density_prob
    return {k: scipy.special.comb(n, k)*p**k*(1-p)**(n-k) for k in range(0, n + 1)}

assert(dispatch_density_key(3, 0.5) == {0: 0.125, 1: 0.375, 2: 0.375, 3: 0.125})
# new version:
# 3 A, c 4, f 4 endu 4
# [0, 0, 0, 1] attacks density
def get_attack_density(weapon):
    assert(isinstance(weapon, Weapon))
    return {a: prob for a, prob in prob_by_roll_result(weapon.a).items()}
# [1/8, 3/8, 3/8 ,1/8] hit density
def get_hits_density(weapon, attack_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(attack_density, dict))
    hits_density = {}
    for a, prob_a in attack_density.items(): 
        # {1: 0.3333333333333333, 2: 0.3333333333333333, 3: 0.3333333333333333}
        for hit_roll, prob_hit_roll in prob_by_roll_result(weapon.hit).items():
            # {5: 1}
            hits_ratio = compute_successes_ratio(hit_roll - weapon.bonuses.to_hit)
            # 0.5
            for hits, prob_hits in dispatch_density_key(a, hits_ratio).items():
                hits_density[hits] = hits_density.get(hits, 0) + prob_hits*prob_hit_roll*prob_a
    return hits_density

# [......]  woud density
def get_wounds_density(weapon, target, hits_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(hits_density, dict))
    wounds_density = {}
    for hits, prob_hits in hits_density.items(): 
        for s_roll, prob_s_roll in prob_by_roll_result(weapon.s).items():
            wounds_ratio = compute_successes_ratio(compute_necessary_wound_roll(s_roll, target.t) - weapon.bonuses.to_wound)
            for wounds, prob_wounds in dispatch_density_key(hits, wounds_ratio).items():
                wounds_density[wounds] = wounds_density.get(wounds, 0) + prob_wounds*prob_s_roll*prob_hits
    return wounds_density

# [......] unsaved wounds density
def get_unsaved_wounds_density(weapon, target, wounds_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(wounds_density, dict))
    unsaved_wounds_density = {}
    for wounds, prob_wounds in wounds_density.items(): 
        for ap_roll, prob_ap_roll in prob_by_roll_result(weapon.ap).items():
            save_roll = target.sv + ap_roll
            if target.invu is not None:
                save_roll = min(save_roll, target.invu)
            unsaved_wounds_ratio = 1 - compute_successes_ratio(save_roll, auto_success_on_6=False)
            for unsaved_wounds, prob_unsaved_wounds in dispatch_density_key(wounds, unsaved_wounds_ratio).items():
                unsaved_wounds_density[unsaved_wounds] = unsaved_wounds_density.get(unsaved_wounds, 0) + prob_unsaved_wounds*prob_ap_roll*prob_wounds
    return unsaved_wounds_density

# last step numeric averaging: damage roll + fnp
def get_avg_figs_fraction_slained(weapon, target, unsaved_wounds_density, N):
    """
    El famoso montecarlo approach
    :param N: number of simulations for each unsaved wounds entry
    """
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(unsaved_wounds_density, dict))
    figs_fraction_slained_total_weighted = 0
    adapted_N = (N // target.w) + 1
    for unsaved_wounds, prob_unsaved_wounds in unsaved_wounds_density.items():
        figs_fraction_slained_total_sum = 0
        for _ in range(adapted_N):
            figs_fraction_slained_total = 0
            for start_health in range(1, target.w + 1):
                n_figs_slained = 0
                remaining_health = start_health
                for wound in range(unsaved_wounds):
                    damages = weapon.d.roll()
                    if target.fnp is not None:
                        for damage in range(damages):
                            if roll_D6() >= target.fnp:
                                damages -= 1  # fnp success
                    remaining_health -= damages
                    if remaining_health <= 0:
                        n_figs_slained += 1
                        remaining_health = target.w
                # e.g. remaining = 1,slained 2, w=3, start = 2: frac = 2 - (1 - 2/3) + (1 - 1/3) 
                first_slain_fraction = start_health/target.w
                remaining_fraction = remaining_health/target.w
                figs_fraction_slained_total += \
                    n_figs_slained - (1 - first_slain_fraction) + (1 - remaining_fraction)

            figs_fraction_slained_total_sum += figs_fraction_slained_total

        figs_fraction_slained_total_weighted += \
            prob_unsaved_wounds * figs_fraction_slained_total_sum/(adapted_N*target.w) # target_wounds is here because of start_health loop

    return figs_fraction_slained_total_weighted

def score_weapon_on_target(w, t, N):
    """
    avg_figs_fraction_slained by point
    """
    a_d = get_attack_density(w)
    assert(float_eq(sum(a_d.values()), 1))
    h_d = get_hits_density(w, a_d)
    assert(float_eq(sum(h_d.values()), 1))
    w_d = get_wounds_density(w, t, h_d)
    assert(float_eq(sum(w_d.values()), 1))
    uw_d = get_unsaved_wounds_density(w, t, w_d)
    assert(float_eq(sum(uw_d.values()), 1))
    return get_avg_figs_fraction_slained(w, t, uw_d, N)/w.points

# Sv=1 : ignore PA -1
wea = Weapon(hit="4", a="4", s = "4", ap = "1", d="3", bonuses=Bonuses(0, 0), points=120)
wea2 = Weapon(hit="4", a="4", s = "4", ap = "0", d="3", bonuses=Bonuses(0, 0), points=120)
tar = Target(t=4, sv=1, invu=5, fnp=6, w=16)
assert(abs(score_weapon_on_target(wea, tar, 1000) / score_weapon_on_target(wea2, tar, 100) - 1) <= 0.25)

def scores_to_comparison_score(score_a, score_b):
    if score_a > score_b:
        return + (1 - score_b / score_a)
    else:
        return - (1 - score_a / score_b)

assert(scores_to_comparison_score(10000, 1) == 0.9999) 
assert(scores_to_comparison_score(1, 10000) == -0.9999) 
assert(scores_to_comparison_score(1, 1) == 0) 

def y_dims_to_str(l):
    return f"""{l[0]}/{l[1]}/{"-" if l[2] is None else f"{l[2]}+"}"""

def x_dims_to_str(l):
    return  f"""{"-" if l[0] is None else f"{l[0]}+"}/{"-" if l[1] is None else f"{l[1]}+"}"""
def compute_matrix(weapon_a, weapon_b, N):    
    ws_ts_fnps = []
    for w, ts in zip(
        [1,2,3,4,6,8,10,12,16], 
        [
            [2,3,4],
            [2,3,4,5],
            [2,3,4,5,6],
            [2,3,4,5,6],
            [3,4,5,6,8],
            [4,5,6,8],
            [5,6,8],
            [6,8],
            [6,8]
        ]
    ) :
        fnps = [7] if w > 6 else [7, 6, 5]
        for fnp in fnps:
            for t in ts:
                ws_ts_fnps.append((t, w, fnp))
                
    ws_ts_fnps.sort(key=lambda e: e[2] * 10000 - e[0]*100 - e[1])
    ws_ts_fnps = list(map(lambda l: list(map(map_7_to_None, l)), ws_ts_fnps))
    
    print(list(map(y_dims_to_str, ws_ts_fnps)), )
    
    svs = []
    for invu in [2, 3, 4, 5, 6, 7]:
        for sv in range(1, min(invu + 1, 6 + 1)):
            svs.append((sv, invu))
    svs.sort(key=lambda e: -e[0]*10 + -e[1])
    svs = list(map(lambda l: list(map(map_7_to_None, l)), svs))
    
    print(list(map(x_dims_to_str, svs)))
    
    
    return  \
        [
            [
                scores_to_comparison_score(
                    score_weapon_on_target(
                        weapon_a, 
                        Target(t, sv, invu=invu, fnp=fnp, w=w),
                        N), 
                    score_weapon_on_target(
                    weapon_b, 
                    Target(t, sv, invu=invu, fnp=fnp, w=w),
                    N)
                )
                for sv, invu in svs
            ]
            for w, t, fnp in ws_ts_fnps
        ]

In [340]:
# SAG vs Battlewagon under KFF and fnp 6+
sag = Weapon(hit="5", a="2D6", s = "2D6", ap = "5", d="D6", bonuses=Bonuses(0, 0), points=120)
bat = Target(t=8, sv=4, invu=5, fnp=6, w=16)

# SAG vs Battlewagon under KFF and fnp 6+
bolt = Weapon(hit="3", a="2", s = "4", ap = "1", d="1", bonuses=Bonuses(0, 0), points=20)

In [336]:
compute_matrix(sag, bolt, 1000)

['8/6/5+', '6/6/5+', '6/4/5+', '6/3/5+', '5/6/5+', '5/4/5+', '5/3/5+', '5/2/5+', '4/6/5+', '4/4/5+', '4/3/5+', '4/2/5+', '4/1/5+', '3/6/5+', '3/4/5+', '3/3/5+', '3/2/5+', '3/1/5+', '2/4/5+', '2/3/5+', '2/2/5+', '2/1/5+', '8/6/6+', '6/6/6+', '6/4/6+', '6/3/6+', '5/6/6+', '5/4/6+', '5/3/6+', '5/2/6+', '4/6/6+', '4/4/6+', '4/3/6+', '4/2/6+', '4/1/6+', '3/6/6+', '3/4/6+', '3/3/6+', '3/2/6+', '3/1/6+', '2/4/6+', '2/3/6+', '2/2/6+', '2/1/6+', '8/16/-', '8/12/-', '8/10/-', '8/8/-', '8/6/-', '6/16/-', '6/12/-', '6/10/-', '6/8/-', '6/6/-', '6/4/-', '6/3/-', '5/10/-', '5/8/-', '5/6/-', '5/4/-', '5/3/-', '5/2/-', '4/8/-', '4/6/-', '4/4/-', '4/3/-', '4/2/-', '4/1/-', '3/6/-', '3/4/-', '3/3/-', '3/2/-', '3/1/-', '2/4/-', '2/3/-', '2/2/-', '2/1/-']
['6+/-', '6+/6+', '5+/-', '5+/6+', '5+/5+', '4+/-', '4+/6+', '4+/5+', '4+/4+', '3+/-', '3+/6+', '3+/5+', '3+/4+', '3+/3+', '2+/-', '2+/6+', '2+/5+', '2+/4+', '2+/3+', '2+/2+', '1+/-', '1+/6+', '1+/5+', '1+/4+', '1+/3+', '1+/2+']


In [341]:
import numpy as np
%time print(compute_matrix(sag, bolt, 1000))

['8/6/5+', '6/6/5+', '6/4/5+', '6/3/5+', '5/6/5+', '5/4/5+', '5/3/5+', '5/2/5+', '4/6/5+', '4/4/5+', '4/3/5+', '4/2/5+', '4/1/5+', '3/6/5+', '3/4/5+', '3/3/5+', '3/2/5+', '3/1/5+', '2/4/5+', '2/3/5+', '2/2/5+', '2/1/5+', '8/6/6+', '6/6/6+', '6/4/6+', '6/3/6+', '5/6/6+', '5/4/6+', '5/3/6+', '5/2/6+', '4/6/6+', '4/4/6+', '4/3/6+', '4/2/6+', '4/1/6+', '3/6/6+', '3/4/6+', '3/3/6+', '3/2/6+', '3/1/6+', '2/4/6+', '2/3/6+', '2/2/6+', '2/1/6+', '8/16/-', '8/12/-', '8/10/-', '8/8/-', '8/6/-', '6/16/-', '6/12/-', '6/10/-', '6/8/-', '6/6/-', '6/4/-', '6/3/-', '5/10/-', '5/8/-', '5/6/-', '5/4/-', '5/3/-', '5/2/-', '4/8/-', '4/6/-', '4/4/-', '4/3/-', '4/2/-', '4/1/-', '3/6/-', '3/4/-', '3/3/-', '3/2/-', '3/1/-', '2/4/-', '2/3/-', '2/2/-', '2/1/-']
['6+/-', '6+/6+', '5+/-', '5+/6+', '5+/5+', '4+/-', '4+/6+', '4+/5+', '4+/4+', '3+/-', '3+/6+', '3+/5+', '3+/4+', '3+/3+', '2+/-', '2+/6+', '2+/5+', '2+/4+', '2+/3+', '2+/2+', '1+/-', '1+/6+', '1+/5+', '1+/4+', '1+/3+', '1+/2+']
[[0.2900064411566179, 0.32

In [242]:
w=len([1,2,3,4,6,8,10,12,16])
t=len([2,3,4,5,6,8])
svs=len(range(1, 19 + 1))
fnp=len(range(1, 2 + 1))
w*t*svs*fnp  # 3200

2052

In [35]:
w_t_fnp = sum([
    1 * len([2,3,4])*2,      #w=1 # w * t possible *2 if fnp variant
    1 * len([2,3,4,5])*2,    #w=2
    1 * len([2,3,4,5,6])*2,  #w=3
    1 * len([2,3,4,5,6])*2,  #w=4
    1 * len([3,4,5,6,8])*2,#w=6
    1 * len([4,5,6,8]),#w=8
    1 * len([5,6,8]),#w=10
    1 * len([6,8]),#w=16
    1 * len([6,8]) #w=28
])
w_t_fnp

55

In [36]:
svs*w_t_fnp

1045

In [32]:
%timeit score_weapon_on_target(bolt, bat, 200)

1.11 ms ± 5.79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
%timeit score_weapon_on_target(sag, bat, 200)

75.9 ms ± 538 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
1*1045/1000/60

0.017416666666666667

In [ ]:
# put FNP only on endu < 7

In [275]:
score_weapon_on_target(sag, bat, 1000), score_weapon_on_target(bolt, bat, 1000)

(0.014493157072157503, 0.021757435705378997)

In [198]:
w = sag
t = bat

In [190]:
a_d = get_attack_density(w)
a_d

{2: 0.027777777777777776,
 3: 0.05555555555555555,
 4: 0.08333333333333333,
 5: 0.1111111111111111,
 6: 0.1388888888888889,
 7: 0.16666666666666666,
 8: 0.1388888888888889,
 9: 0.1111111111111111,
 10: 0.08333333333333333,
 11: 0.05555555555555555,
 12: 0.027777777777777776}

In [191]:
h_d = get_hits_density(w, a_d)
h_d, sum(h_d.values())

({0: 0.09245826180349491,
  1: 0.2239853530307222,
  2: 0.26620813348361666,
  3: 0.21083128491947162,
  4: 0.12367835752228379,
  5: 0.05596105682474631,
  6: 0.019891044662844356,
  7: 0.005563176345069349,
  8: 0.0012047433299274988,
  9: 0.00019485804737601263,
  10: 2.210969797211731e-05,
  11: 1.5680636859657664e-06,
  12: 5.226878953219221e-08},
 1.0000000000000002)

In [192]:
w_d = get_wounds_density(w, t, h_d)
w_d, sum(w_d.values())

({0: 0.4029500867053239,
  1: 0.3367914778591636,
  2: 0.1689350056321175,
  3: 0.06478956365764589,
  4: 0.020154421512724424,
  5: 0.00512477251439623,
  6: 0.0010565739623414905,
  7: 0.00017367555510426272,
  8: 2.216603219186902e-05,
  9: 2.1100352269821476e-06,
  10: 1.405806713069297e-07,
  11: 5.83937585822815e-09,
  12: 1.1371713614420627e-10},
 1.0000000000000004)

In [193]:
uw_d = get_unsaved_wounds_density(w, t, w_d)
uw_d, sum(uw_d.values())

({0: 0.5366555241737849,
  1: 0.3162275735568333,
  2: 0.11078687649773596,
  3: 0.029102035242819145,
  4: 0.006064610751263366,
  5: 0.0010130132279732571,
  6: 0.00013499857838335794,
  7: 1.415632468850579e-05,
  8: 1.1407205971613996e-06,
  9: 6.802863136706233e-08,
  10: 2.8236457380877304e-09,
  11: 7.27678850725364e-11,
  12: 8.764573859500293e-13},
 1.0000000000000007)

In [284]:
l = 
[[-0.7148148148148149, -0.7148148148148148, -0.6577777777777779, -0.7148148148148148, -0.714814814814815, -0.5722222222222224, -0.6435185185185186, -0.714814814814815, -0.7148148148148148, -0.4296296296296298, -0.5246913580246912, -0.6197530864197534, -0.7148148148148148, -0.7148148148148149, -0.14444444444444482, -0.287037037037037, -0.4296296296296299, -0.5722222222222223, -0.7148148148148149, -0.7148148148148148, 0.298701298701299, 0.298701298701299, 0.12337662337662325, -0.14444444444444426, -0.4296296296296296, -0.7148148148148148], [-0.5437013102050048, -0.5504726746152665, -0.45607374121762545, -0.5498307729461709, -0.5531253486334178, -0.31856815140470274, -0.43751237113853725, -0.5571768688233366, -0.5609560232367858, -0.09055547226072969, -0.25260355383448996, -0.4054993036631931, -0.56382492531665, -0.5714246808849908, 0.2661972912188242, 0.11145907494404672, -0.11274484377962668, -0.3405782300400615, -0.5677708506274355, -0.5836731840625873, 0.5549352165073247, 0.5564119431179052, 0.43690155913272444, 0.23612737105033021, -0.1454250000972267, -0.5831209147697345], [-0.42257241545654123, -0.43252955332525556, -0.3037995050774931, -0.4275311399897842, -0.43409304054862363, -0.13363482790521242, -0.27935811468352423, -0.43640591250317673, -0.4471338844836169, 0.13919851772896175, -0.046586225447518004, -0.24429492800620312, -0.43647536763924943, -0.45443309538000376, 0.42563685147054786, 0.3035445818812015, 0.1182869347343507, -0.16640071358145359, -0.4532884939998255, -0.4744693926641982, 0.652132926027015, 0.6507261960768943, 0.5578848356480364, 0.402403956770515, 0.08411230431070715, -0.47337395307201613], [-0.32556669253618375, -0.3349624834696705, -0.19818217923979697, -0.3341397092617501, -0.3395432555055693, 0.0029336542370221874, -0.16163738942123085, -0.34313787844168653, -0.3558544379084627, 0.25612899848508075, 0.09178656123739937, -0.1257761032197643, -0.3514213307159828, -0.37275381875805624, 0.508211061174811, 0.39877173224225615, 0.23588952102884786, -0.03167074145180393, -0.3654402637768748, -0.3769020114033207, 0.6992720568840646, 0.6995736570418687, 0.6172494074775884, 0.4842839336630096, 0.19845395287755385, -0.3836379427728802], [-0.6637731481481481, -0.6637731481481481, -0.5965277777777778, -0.6637731481481481, -0.6637731481481484, -0.49565972222222243, -0.5797164351851853, -0.6637731481481484, -0.6637731481481481, -0.3275462962962964, -0.4396219135802468, -0.5516975308641976, -0.6637731481481481, -0.6637731481481483, 0.008605851979345602, -0.15943287037037057, -0.3275462962962966, -0.4956597222222222, -0.6637731481481483, -0.6637731481481481, 0.40516351118760785, 0.40516351118760785, 0.25645438898450956, 0.00860585197934649, -0.32754629629629606, -0.6637731481481481], [-0.46339483696145733, -0.4688542316593808, -0.36052641245589756, -0.4703367489252799, -0.47594838332165723, -0.19537514882517215, -0.3370817172584911, -0.4780774800828492, -0.48766386797367256, 0.06701474403763852, -0.11518433698861263, -0.30607961858470345, -0.4878360665401801, -0.497792351174286, 0.37954191686543026, 0.2451650856390929, 0.04475730672927025, -0.22695770127320658, -0.4966658735079167, -0.507090259095732, 0.6223239446320938, 0.6225417131709704, 0.5218614107789683, 0.35101008076998264, 0.0037591337439875083, -0.5071760718315839], [-0.31490239481899374, -0.31711200294935815, -0.17593491105674464, -0.32652703750038703, -0.3333782647789375, 0.0272464669179997, -0.153263244554791, -0.3361125727214601, -0.3496508216331836, 0.26957494513558156, 0.10780327164609227, -0.11420306839647032, -0.34617288656441936, -0.36435629868027053, 0.5100375274969915, 0.40577706894820575, 0.24721295314406377, -0.0152659385419609, -0.3580480807245918, -0.3873171864335041, 0.7008204100235347, 0.704821624292669, 0.6259172426666357, 0.4898722095253204, 0.2123198217563641, -0.3846835725707417], [-0.2090560462140666, -0.21705658247962767, -0.05529605011132799, -0.21890764844478217, -0.2193001549310838, 0.15600667981502436, -0.025683774137923443, -0.22726311982243108, -0.2517939024130732, 0.36651990973586335, 0.23414338875597174, 0.03720257166037866, -0.2396542479654551, -0.24905641383496468, 0.579500965200793, 0.48581852721267205, 0.35417852835525065, 0.12124602635879189, -0.2524611771976003, -0.2675940727874587, 0.7436297275921254, 0.7434441291059222, 0.6754186047280837, 0.5636227535873153, 0.334093946648958, -0.27645569812903736], [-0.09949688374688015, -0.11149490323285349, 0.07308213648451134, -0.09737893137270681, -0.10815091875649174, 0.2509655526232526, 0.10703871818120847, -0.1142181211291351, -0.12445252081029856, 0.4460471765828016, 0.32678442362601046, 0.16464870326294112, -0.12009156920623676, -0.12729545098134676, 0.6307801534724391, 0.5537760653727408, 0.43517497689484586, 0.25550366257544754, -0.12745310891592798, -0.1402378732985451, 0.777945806882188, 0.7799688237648228, 0.7208029524528097, 0.6249359748616262, 0.4213010454687691, -0.13973143873889882], [-0.597608024691358, -0.597608024691358, -0.5171296296296297, -0.597608024691358, -0.597608024691358, -0.3964120370370371, -0.4970100308641976, -0.597608024691358, -0.5976080246913582, -0.1952160493827162, -0.32934670781893016, -0.463477366255144, -0.5976080246913582, -0.5976080246913581, 0.17162032598274202, 0.005944391179290465, -0.19521604938271586, -0.3964120370370372, -0.5976080246913581, -0.5976080246913582, 0.5029721955896453, 0.5029721955896453, 0.37871524448705673, 0.17162032598274202, -0.19521604938271608, -0.5976080246913582], [-0.3649015010970602, -0.37091701732750215, -0.23775774378712555, -0.3691545879781336, -0.37806741350337225, -0.04718768458912692, -0.21359906275389928, -0.37850091536935704, -0.38687319442906565, 0.21535286769927087, 0.05168183276191973, -0.16727209555982658, -0.382523249592545, -0.39732300186731584, 0.4764213349065278, 0.36491754151026734, 0.197673551652495, -0.07718548094668687, -0.3967440410806474, -0.41070141440133323, 0.6841891540330545, 0.6837890447621615, 0.5991717383954143, 0.4562032771562414, 0.16695973082559845, -0.4128291161795957], [-0.19276931192194735, -0.1953500888745172, -0.025602419176052993, -0.19884918802414497, -0.21147330097618833, 0.18076084874264087, -0.004489600385288672, -0.2081186770776604, -0.22305553745891193, 0.385799527641734, 0.2535896222161824, 0.051944989842474976, -0.22533699949763253, -0.23856894744482848, 0.5882242241610635, 0.5053964199527328, 0.3647001280763106, 0.14897387510558024, -0.24076092602993648, -0.2612303422265455, 0.7521800194441525, 0.7499299134599142, 0.6818252360816723, 0.5724455744782126, 0.3420722147166826, -0.25835727898722416], [-0.06032948016304529, -0.06746967281575966, 0.12144370313075403, -0.07136837104196858, -0.0822266719365855, 0.2839273413942025, 0.13979194693687957, -0.07809372291777072, -0.10026794344836498, 0.4687518765777937, 0.35186302853706075, 0.18390450898631838, -0.09101412110275953, -0.11230583432873886, 0.6450839729461936, 0.5692380565257243, 0.45131208752546126, 0.2622259535358623, -0.1143429171211725, -0.13523324008223414, 0.784946169221325, 0.7840737440438699, 0.7269440943687939, 0.6302389094565465, 0.43617776439170985, -0.14159121134108887], [0.06568630906633177, 0.059450450853543946, 0.2230549372751146, 0.057867626146457685, 0.0564060576990284, 0.37622383984520114, 0.24615385866236716, 0.05140226360402711, 0.06598377580073478, 0.5350268457339576, 0.44031864907805496, 0.29358728338327866, 0.059070360013103285, 0.032148111879605534, 0.6910690629540392, 0.6253708933084681, 0.5280587701489358, 0.369725266946882, 0.04819499913898617, 0.00945118901233144, 0.8124855695528935, 0.8135633915282887, 0.7642919849787427, 0.6858476717985285, 0.5185544234269137, 0.04174228111987355], [0.11990562254845438, 0.11855744399312029, 0.25953659295815457, 0.1191723829651582, 0.11662014157925793, 0.4164810315146089, 0.29888097182213047, 0.11043223106993316, 0.10519268124578074, 0.5640971230614188, 0.4658400897438586, 0.3399184297374569, 0.10084188309862785, 0.105988488378982, 0.7063218030602412, 0.648585888960226, 0.557924079590564, 0.4003561762573128, 0.11040686539036648, 0.10588712085032304, 0.8221363564957135, 0.8238373786869866, 0.7790462036449011, 0.705768040992391, 0.5559219291848979, 0.11122341436677985], [-0.14955453961256904, -0.1568157966564624, 0.01289138375978438, -0.16539296553983407, -0.1668689928485162, 0.21252776716160382, 0.04692839097838064, -0.17084398572447002, -0.1831312126005714, 0.40476696050631256, 0.2855087834862, 0.0926483923125947, -0.18504965739075174, -0.19378060265489938, 0.6050419921952154, 0.5221909149254751, 0.3974886170294444, 0.17956016646238926, -0.19998580429492296, -0.21564963250304958, 0.762775161028061, 0.7620086111367439, 0.7005584857047761, 0.591360383227743, 0.3736860509716007, -0.21912030987802822], [0.07816403578419229, 0.057801077825359926, 0.22858517604812034, 0.06731425969556115, 0.052271217668842684, 0.3841399368233085, 0.2448943360952236, 0.0571436059526389, 0.036707760384444454, 0.5358478558354118, 0.4384040072426678, 0.28969279822072114, 0.040730330011282345, 0.025712269076305416, 0.6911882425742116, 0.6265592431577754, 0.5245276579664979, 0.3539780445214227, 0.018369192046839067, -0.011517771895402706, 0.8140313603807099, 0.812918620901772, 0.7641213777710256, 0.678254149891162, 0.5071949370283626, -0.014428074728781604], [0.20098043897879259, 0.19761999716982648, 0.3321162399910773, 0.18596983966678082, 0.18492960473806552, 0.4693042084112685, 0.3502873279839498, 0.1790670140348385, 0.17391954279399413, 0.603266829867944, 0.5173430123160865, 0.38693404160486267, 0.17207278349899935, 0.14623011836140498, 0.7356153834062067, 0.6753373374814297, 0.594071294562259, 0.4481237645296, 0.15602620634299225, 0.14051498201370383, 0.8397766511262419, 0.837155381143335, 0.7977464581572808, 0.7236876955038629, 0.5771543878837354, 0.1392767869743331], [0.30762166655724443, 0.30067224156678596, 0.42015289934568123, 0.2999474245912478, 0.29155425089504494, 0.5358922564376123, 0.4413980208735332, 0.301301094254117, 0.2899933312739108, 0.650639489676692, 0.5811532047249728, 0.4743902850694166, 0.28260739470104324, 0.28228004634518833, 0.7685897057439312, 0.720807203577392, 0.6501262084603765, 0.5275466872061167, 0.2907677599198373, 0.2679306809769755, 0.8602708161796611, 0.8604621267898171, 0.8232363877061921, 0.7625664151277599, 0.6371718670642699, 0.2698675254180607], [0.3410159534908034, 0.34919639385967094, 0.454474807089545, 0.33185490803992, 0.3368826917274028, 0.5591270323487845, 0.47271383135817446, 0.33593227004737647, 0.3320783084927077, 0.6698319080009301, 0.6033706237138343, 0.5004139323543764, 0.34073760277188636, 0.3417495089217105, 0.7808116724010344, 0.7394779506444553, 0.6688796473957608, 0.5569857918993956, 0.3373689075061129, 0.3383642913943252, 0.867448479796045, 0.8693993232362768, 0.8350432186161817, 0.7785101622738352, 0.6679218735885043, 0.33392831133316425], [0.37150857240026924, 0.36727820880549555, 0.47321832007969156, 0.36891818611800353, 0.3692345401746071, 0.5793215732311753, 0.5006492562471838, 0.36898658267268236, 0.3626295757341401, 0.6831877983631964, 0.6201670280162638, 0.5226699656326939, 0.36478456913026347, 0.36140335836792825, 0.7897535056432345, 0.7475981557911404, 0.6811668574249237, 0.5694011219760491, 0.3643460413945292, 0.35481862358010674, 0.8733109217515744, 0.8732277631852132, 0.8438681827612763, 0.7869118094174232, 0.6820785121059587, 0.37114580772559347], [-0.05533114608117862, -0.06800281518746154, 0.12015146278200983, -0.06510826835638739, -0.07996808720068593, 0.2950032589881455, 0.1453330855256304, -0.07909528969867685, -0.09523821445217096, 0.47021136359185356, 0.35666665988651847, 0.18233651487290403, -0.09705240169733875, -0.11563963298531488, 0.6463301277675438, 0.5714720502398201, 0.4582524996120865, 0.2620891798230156, -0.11178115528899002, -0.1259162441151238, 0.7867922479743675, 0.7845043994246256, 0.7305499038845231, 0.6354684441236144, 0.44257340943593093, -0.12675766785271947], [0.0939575263455581, 0.06653000743036797, 0.23393197851017444, 0.0769946416409436, 0.05941300647412173, 0.39495787522892145, 0.2626694487260184, 0.06128485892397095, 0.05897147878734332, 0.5437276005494484, 0.4509750930501052, 0.30148324399984394, 0.04749398845922059, 0.03613204643277079, 0.6921712928813288, 0.6289941663617917, 0.5364715726783793, 0.3708094888674962, 0.041607618028124516, -0.0031120695161852563, 0.815816509913899, 0.8149918663233293, 0.7678679283773695, 0.6825937287575258, 0.5183551339982364, 0.016804864420765786], [0.20815800472395296, 0.19500541962381512, 0.3396307756660889, 0.20894185982044233, 0.18985399665929703, 0.47199833753256126, 0.3631358547697947, 0.197964326399238, 0.19328107648354464, 0.6027551472208188, 0.5234383590639918, 0.3932196062322272, 0.19380216454829702, 0.19157735472816118, 0.7355300418059374, 0.6813563841903237, 0.5987340715661098, 0.45844939988047295, 0.19406506614788666, 0.1774651406184392, 0.8407061170481356, 0.8392988705836486, 0.7990944253972934, 0.732137975892171, 0.5908971767238035, 0.1807960784023196], [0.25435700641350967, 0.24438777125186295, 0.3791140570435041, 0.2474101571306736, 0.2474302999263276, 0.49963975657114024, 0.3919522046352264, 0.24106677355116823, 0.24076022377279438, 0.6264685780791144, 0.5460863187325413, 0.43248349045505863, 0.2410188070117285, 0.24279869655385, 0.7491072784694512, 0.6992974553309774, 0.6239611498219135, 0.49930099036009, 0.24321494918812536, 0.24223845902978203, 0.8493116900111725, 0.8510497835267457, 0.8115838373349494, 0.7479826506412945, 0.6252176881135816, 0.23706968217785895], [0.2811548027764016, 0.27719065230811235, 0.4033549997022349, 0.28045687254033724, 0.2828865128877539, 0.5202871349833524, 0.426704872338467, 0.2728799954291492, 0.2824920472349737, 0.6395122612667368, 0.5691606227334162, 0.4632995388935569, 0.2793354850825239, 0.2689062160968597, 0.7618118899167342, 0.7153088540977964, 0.6392528790203728, 0.5201759266727914, 0.2878108004897175, 0.2790972891947492, 0.8547950331104137, 0.8577020093714071, 0.8202337439084371, 0.7568941851319755, 0.6390409052336721, 0.2899143836361151], [0.31245263393324285, 0.30311083655041426, 0.4272760193656163, 0.3012491084879657, 0.3105358851714265, 0.534162957715494, 0.4416868587743421, 0.30382276169389644, 0.3079421058686467, 0.6527907120477026, 0.5816854084178423, 0.48273877191477177, 0.30030861910408424, 0.3118826070937186, 0.7675091028754646, 0.7215212362556631, 0.6482976057616185, 0.5324191566538113, 0.3023160034098621, 0.29341313295338933, 0.8595517928098317, 0.8611036934275811, 0.8264251368489557, 0.7644759991868983, 0.6499715185427641, 0.3169191857113771], [0.32640995210011514, 0.3298318688838676, 0.4398465866066762, 0.32642328564013157, 0.3301491646585569, 0.5510504132041092, 0.4646314493208985, 0.32609547795177873, 0.33148656076102534, 0.6627022317919833, 0.5927381867695152, 0.4966199294328477, 0.32622854563577663, 0.3314598527280258, 0.7766706628206477, 0.730944222917779, 0.6695121480996963, 0.553278064214238, 0.33769517238412683, 0.319770873168268, 0.8659212072338905, 0.8656573011076091, 0.8309533646210959, 0.7775137387562341, 0.6596845880836173, 0.32686517433468354], [0.4906879831114599, 0.49083326108268055, 0.5683478037582492, 0.48858945039025203, 0.48346993489533385, 0.6588283884443994, 0.5870370860471728, 0.48513933768682505, 0.48150463537866717, 0.7456478504016377, 0.6920958806517138, 0.6090688753304729, 0.4748594584503101, 0.47525546182124967, 0.8307428720960883, 0.7929417748250678, 0.743893249345603, 0.6510667879049005, 0.4748724999725543, 0.46249010336551666, 0.8974715743191558, 0.8958368328799123, 0.8685105472647598, 0.8253510300444972, 0.734850849709904, 0.4697400863817647], [0.5149065838562485, 0.5129666767908622, 0.5937542837879239, 0.5160183834042942, 0.5139279629981495, 0.6771128646560582, 0.6069644998663546, 0.5158589031306053, 0.5150511873514061, 0.7595684951675332, 0.7048465179735738, 0.6380811350373752, 0.5140755567687092, 0.5102203287340507, 0.8388439958945568, 0.8088187584248908, 0.7564107906631412, 0.6778140138886588, 0.5121951285597865, 0.5124668562931883, 0.9032986095284166, 0.9037431900108963, 0.8794969524259878, 0.8395100968763426, 0.7559466336833784, 0.5027547063161926], [0.5376830093266078, 0.5410915236156062, 0.6181804275096671, 0.5364268920137873, 0.5399113482098299, 0.6944203676660654, 0.6268595770247516, 0.5258146243886698, 0.5305345052547803, 0.7696180481828128, 0.7201810516941869, 0.6519032071033055, 0.5412203815565709, 0.5442264128887405, 0.8471361960572016, 0.813856377650623, 0.7664650059619131, 0.689761068310069, 0.5395336140072319, 0.5367183032016729, 0.9075062264398167, 0.907174731075614, 0.8847829098052855, 0.8466061179497788, 0.7693466546294516, 0.5340098450908815], [0.5537124051744039, 0.5434093817135681, 0.6265911998313505, 0.5498055778328754, 0.5470022213140364, 0.6987539785425088, 0.6431349471645884, 0.5474730037360378, 0.5501600593102604, 0.7768569217424561, 0.7279671850029863, 0.6595810132368659, 0.5544914471941654, 0.5452637994741497, 0.8494290763317808, 0.8194615520116317, 0.7725835383104107, 0.7001655533617777, 0.5506524979881868, 0.543414780563698, 0.9100340875534516, 0.9094507783633285, 0.8868674544017138, 0.8499639829022178, 0.7792158637162576, 0.5591797565393467], [0.5662383523492704, 0.5681364517737488, 0.6394101779132577, 0.5684101352664617, 0.5641032519844481, 0.7084506353298101, 0.6567356136642815, 0.5675103555927631, 0.5672038752852381, 0.7861952681370415, 0.7442887377937619, 0.6758060103739036, 0.5633503437788532, 0.5698367948141974, 0.8565807513332042, 0.8282316687710333, 0.7868728277275034, 0.7116355888695403, 0.5593646234447156, 0.574681019895269, 0.9135761397270115, 0.9138523960219079, 0.8909877996004347, 0.8537446524270838, 0.7823975336163844, 0.5675013282331502], [-0.6657270448508611, -0.6640694574785333, -0.6083841218080961, -0.670863769671901, -0.662630860286127, -0.49560707720074315, -0.587940633682981, -0.6665213596399324, -0.6791743684603675, -0.3360211836807885, -0.4542246731479237, -0.5587331064280867, -0.6610162921925752, -0.6652747500213503, -0.015507825509403217, -0.16647499426508083, -0.34064377808979807, -0.4991343079469681, -0.6716930949514116, -0.671248096828098, 0.39655238162509876, 0.3940520403709641, 0.2463838323000277, -0.009603637329726467, -0.33320314565752385, -0.6663948989624292], [-0.4910921594700175, -0.4881417072632547, -0.39213808174933407, -0.49251909023779195, -0.4897596090147691, -0.2315720612023896, -0.36689005223577953, -0.49663680737789606, -0.49960023362847894, 0.01571988108087885, -0.14875548513023573, -0.31571416854164924, -0.4946140693853739, -0.5225881126059222, 0.364937469594659, 0.1914706962963092, 0.013285543138111144, -0.25733299449664293, -0.5077528533219664, -0.5247301951003138, 0.6068722823996135, 0.6076077139612626, 0.4978099597878556, 0.33859878723705084, -0.0236276694537777, -0.5207017658598201], [-0.3666301440438924, -0.36902694635247246, -0.2263475497928964, -0.35703825288549407, -0.3806811747819808, -0.03995240660118404, -0.21142805758859173, -0.369574174131177, -0.4013111976610312, 0.22749138771576094, 0.062072875620816315, -0.17776769154283345, -0.38292865242372287, -0.3972286764427895, 0.4742866584800932, 0.3678858466470253, 0.19841322323509303, -0.07993768718426819, -0.3980878130650003, -0.40603864617802443, 0.684060632023385, 0.6856384926940053, 0.5975061674537914, 0.4647948517011159, 0.1935748689302521, -0.41638814176217787], [-0.26804521195378894, -0.28892308033062164, -0.13257208106428564, -0.2737362779429888, -0.29783666421530064, 0.08567140006635221, -0.0961064956342168, -0.3017829851248621, -0.3019142128071789, 0.28972170468253966, 0.1410912562830935, -0.047595061427139096, -0.3018761642678559, -0.31810519197666676, 0.5349484756359015, 0.4249853897134518, 0.29686851192397656, 0.01829940851168199, -0.3180470999603481, -0.3336812319637248, 0.7140519041534299, 0.7236291397010619, 0.6473895277275905, 0.5150071915002257, 0.24161435095270245, -0.31995313846950835], [-0.6112226667621361, -0.6144655216938859, -0.5313927667258495, -0.6133847507902936, -0.6137914072465823, -0.4081488341697159, -0.5043864168733538, -0.6063791170361806, -0.6091634660440103, -0.21842770928119437, -0.3392134349421073, -0.47081047435720036, -0.6081939720769037, -0.6087304970559972, 0.14553824426521578, -0.03309914622519283, -0.22228857767137533, -0.40107919665724756, -0.6097141339943859, -0.611575962009928, 0.4794179043138864, 0.49123942271935583, 0.36735693426398286, 0.16657270890277953, -0.21380711669667052, -0.6146100896220192], [-0.39385512773696374, -0.39865872970643523, -0.2674943854266596, -0.400030123462512, -0.4104394648753552, -0.11122900738636587, -0.256368819941313, -0.41517949573866797, -0.40906511720949135, 0.17378019687959467, -0.00991827680919899, -0.1997039258490667, -0.4242721905247949, -0.43036509364316355, 0.45010359887721874, 0.33945869996975064, 0.14292365632677695, -0.12359424240956296, -0.4258182979140033, -0.43256743241595597, 0.6680246971075183, 0.6620222587966289, 0.5735254995245959, 0.4235887421773026, 0.12080954692720636, -0.42526833578707013], [-0.2564601869865918, -0.25960483451073224, -0.08520809816270258, -0.2545845772947892, -0.2688479489334462, 0.10767756503339654, -0.07452927898253403, -0.2666292647637736, -0.28917634099053613, 0.34857115620063095, 0.19524530468518886, -0.015207817071160479, -0.2639700405562543, -0.2905001212137469, 0.5549632595100168, 0.47275130768346807, 0.305492060199545, 0.08235277020681786, -0.29766803903558503, -0.3197045066296207, 0.7300539656128336, 0.7298391012462666, 0.6617908302430002, 0.5266843452919928, 0.3043329932643005, -0.31731841405402317], [-0.15768010148253975, -0.1570168464703423, 0.012453844237223288, -0.14743780029639253, -0.15735340930262742, 0.21086919186706143, 0.052990471592345934, -0.157895486237274, -0.19341237051159443, 0.4017209979411601, 0.283405869362939, 0.0971630403173469, -0.1772491468438746, -0.19016344113319328, 0.6013197669053164, 0.5121050289094702, 0.39313267918983896, 0.18039299853255986, -0.19330176508408226, -0.23662470116809742, 0.7586019060553406, 0.7553388425765736, 0.6980676230880223, 0.5932982715644788, 0.3690467062661186, -0.21711225721982708], [-0.0610373799195606, -0.08893614895097968, 0.09268567294356456, -0.07382067051963792, -0.07110473890119928, 0.28443363979980063, 0.14507771177091988, -0.07125693280720957, -0.08729519370956118, 0.4539285075525421, 0.3668987833347056, 0.19068165922373692, -0.0786666480573106, -0.09341361174885365, 0.6374922441961859, 0.5679300784015473, 0.44764855676766435, 0.27114803644678354, -0.07156235935957389, -0.10058148889962171, 0.7827679984189431, 0.7824717035296644, 0.7300159178762302, 0.6428042466224009, 0.4700332228472882, -0.10028486296712691], [-0.5316883513140322, -0.5214891997585647, -0.4405807634873955, -0.5354328150437542, -0.5338630900906146, -0.29233338489857363, -0.4168893947321821, -0.5311708853523739, -0.5352887340130701, -0.07089154330520098, -0.21575554251005136, -0.3770860592319366, -0.5322676668907989, -0.525915356382528, 0.27341691345027475, 0.15243678089691437, -0.07127638564868777, -0.28207949348301653, -0.5282673248141312, -0.534835629927018, 0.5719106835803585, 0.5755710889615215, 0.4660675507217499, 0.30613781988681066, -0.04900148351903866, -0.5303763126006134], [-0.287194082399724, -0.2935856062525595, -0.13628952292809615, -0.28667581990083024, -0.3135248246660458, 0.08006808026672674, -0.10884243626296009, -0.29425807182433295, -0.306736785985841, 0.3167428739191306, 0.17246281434683208, -0.04064811839158977, -0.32544738135647777, -0.2998324900725452, 0.5375410564070134, 0.4459002716987369, 0.28516216635374525, 0.03946003915673291, -0.33252065153126287, -0.33421412467453837, 0.7186114576435936, 0.7230499732384825, 0.6507808431011789, 0.5224196168626618, 0.26440805946296564, -0.3373881337299445], [-0.1032890829062666, -0.10304495741018527, 0.07831226718272866, -0.11544662275252282, -0.13743716599014288, 0.2526307539045347, 0.1028669385765183, -0.13978187964412536, -0.14117046449004012, 0.45544960074106, 0.3375389237661558, 0.14096282440872732, -0.15440873882637418, -0.17442269524177523, 0.6189664457416282, 0.5598274373839205, 0.41902406111685075, 0.23064156112438916, -0.1658428797292001, -0.1808165609421526, 0.7792679142856969, 0.7720745039537693, 0.7213400324924514, 0.6189896944693913, 0.3996214864847737, -0.16123416163486648], [0.007479173690050689, -0.007830960572397738, 0.17990523562601834, -0.00632743688195192, -0.009659561416044515, 0.331000572613524, 0.20568547805416293, -0.005864382779310673, -0.01918174961960395, 0.5011260813832763, 0.4156188042727301, 0.22924042122020571, -0.0381245869606236, -0.04308709405736211, 0.6703461812680176, 0.596534532668642, 0.49145617490022975, 0.31335573150775187, -0.03870655692592839, -0.07146340693387543, 0.801037327632284, 0.800452105401731, 0.7459221236173286, 0.6650053172880614, 0.48526759328621594, -0.06424088282514384], [0.10433256147909487, 0.08991295155308321, 0.24883049223545917, 0.1088656157769542, 0.10266194150494701, 0.39900228594792364, 0.28353002866358856, 0.09535550422820849, 0.10526046436309544, 0.5438331342075047, 0.46346747969112867, 0.3285085968971241, 0.11010038691760238, 0.11575078084950852, 0.6988655063280834, 0.6394052420004155, 0.5481762399574561, 0.3980976119019505, 0.053021564385493436, 0.08814077211180793, 0.8180568484994913, 0.819072330463031, 0.7789311660448135, 0.6917098756490733, 0.5412179840156586, 0.09532283574554612], [-0.047130481975406036, -0.023955236529619284, 0.11442108974606457, -0.06127011635576107, -0.05773543549226856, 0.3032543289533076, 0.16556852384770826, -0.055128526553863466, -0.08379525777050079, 0.4785746009839833, 0.36871687872640235, 0.20928048901891105, -0.07283760547815743, -0.08590005901455733, 0.656157910174598, 0.5769126221519056, 0.46582368115353945, 0.26777422367075665, -0.09332447340137096, -0.06032626692370824, 0.7934000828460512, 0.7897473805347739, 0.7290781426980129, 0.6433294488405757, 0.45375103807588835, -0.11101915759741365], [0.16146710153386845, 0.13218741719820648, 0.30266630653564, 0.15609763408698796, 0.15092638340746856, 0.43473178044249605, 0.3282536529337985, 0.14026740360816803, 0.1491418977593153, 0.5685299079276002, 0.4928970372982945, 0.35565221121036317, 0.12990309087375362, 0.12349222463295018, 0.723206300416462, 0.6656345235336386, 0.5685187949815583, 0.4314066982973429, 0.13839082875786957, 0.09467365557599583, 0.8304592943485748, 0.8281531911188247, 0.787142806697492, 0.7106681146389575, 0.5554265179627909, 0.09144839438649277], [0.2674754745377509, 0.23930406176734775, 0.36844035803834463, 0.23782341911842053, 0.23917012009338356, 0.5038697843015891, 0.39138091357897664, 0.2540549780563881, 0.24193758808498844, 0.6240945449892707, 0.5439493477800927, 0.4151794620594321, 0.24113351596213817, 0.24011294860606136, 0.7521351642608782, 0.6942310711478774, 0.6141856905001781, 0.49053650421800765, 0.22269900313819702, 0.2085517216832139, 0.8494383943586542, 0.8465058601362069, 0.8059420679775318, 0.7479499000358163, 0.6053306957810176, 0.20916735681214682], [0.32227788393645096, 0.32057470472727256, 0.452868386072092, 0.3226632252576801, 0.31490886711268884, 0.5448938894499548, 0.44908396227281167, 0.33308415066141406, 0.3295134832329305, 0.650437687872651, 0.5983451913938291, 0.4947835052816436, 0.3330018866685258, 0.3160343582085573, 0.7769088924763453, 0.7357747511712147, 0.665510506026301, 0.5500096634538978, 0.32117036196459103, 0.3182016479794886, 0.861486214697204, 0.8653283563376005, 0.8321417560590922, 0.7780169053326466, 0.6639120168790349, 0.29963101828985217], [0.04773529489610617, 0.029016431376521434, 0.18971259640678328, 0.027033718739608914, 0.03418132310006616, 0.35659866607792823, 0.21569236405733694, 0.017582291805547934, -0.003977443410201453, 0.5286077281371763, 0.41394373561708186, 0.27402338223222156, 0.002220530827934719, -0.01887171336255189, 0.6772234060876755, 0.6113299093284992, 0.5095675619174457, 0.3407433913123433, -0.04233775835890452, -0.013403083096625745, 0.807589024045044, 0.8058361963013712, 0.7537001300047423, 0.6700403410442539, 0.5049370978558806, -0.026981194883208937], [0.13525110585687328, 0.13258064169001726, 0.27892441120026323, 0.13241699118017647, 0.13048786428598036, 0.43369510996373095, 0.3172450414023583, 0.15689796100681097, 0.13392242476790606, 0.5820609604478151, 0.4878839074782453, 0.33472444401798407, 0.13228416950640565, 0.10928704002654954, 0.7195040898789793, 0.6540165578569063, 0.5672546520692423, 0.4250973447522295, 0.11426082850373664, 0.08076105595108463, 0.8300882008468633, 0.8275468938573246, 0.7848768731843301, 0.703901416341501, 0.5651935806368467, 0.1000652337161464], [0.22004080559458128, 0.24327782027643585, 0.34503167848197525, 0.2358170670288644, 0.2233178512919609, 0.502287684857988, 0.38190754622077394, 0.23059594975498143, 0.2274243682142788, 0.6189288139803165, 0.5409599045501847, 0.4379740625713877, 0.22719334563310978, 0.19560030781547, 0.7409524757541449, 0.7031075328137035, 0.6124912918695651, 0.49599005948921115, 0.210909056911611, 0.23528080202815138, 0.8441229115874251, 0.8437324514367501, 0.8077592538648091, 0.741400298333664, 0.6063994220318787, 0.20101715205190396], [0.4935582046951186, 0.501122829342717, 0.5852692968263553, 0.5050745659563942, 0.49561026693031773, 0.6654584884191342, 0.6089229150488413, 0.5199505552929498, 0.4953969035605582, 0.7523460808326073, 0.7094060309630825, 0.6227823264772927, 0.4892318781891011, 0.5083341276496638, 0.8318241941222131, 0.8042011407771943, 0.7500211071230024, 0.6566914237527073, 0.49673645251445875, 0.5118704810914594, 0.9021419867459888, 0.9004808329952032, 0.8754380202568425, 0.8318608062984321, 0.746470887301407, 0.5092828868132049], [-0.6065396937497634, -0.6038639685254413, -0.5240225298176, -0.6158290903304628, -0.5996062655597448, -0.41824162462102854, -0.5064042177640573, -0.609192973743112, -0.6176930863803638, -0.2398204649570782, -0.33635373811624214, -0.4842268714286053, -0.6086891111460013, -0.6073771487925319, 0.131042156295603, -0.006747205232176201, -0.2164517914852968, -0.400701528387987, -0.6114067137989453, -0.6058891682079703, 0.5153471045586706, 0.4821189969954711, 0.35673096540077376, 0.13684111306770486, -0.2038703049191517, -0.6043635786472308], [-0.4322491011303179, -0.43492059200272926, -0.2873368837057406, -0.42289372267911596, -0.4182702742798756, -0.11569625807386463, -0.267669159719676, -0.4234156355103572, -0.44010032706878766, 0.1433490180356446, -0.038078125415239694, -0.2509717610068073, -0.4288676448053872, -0.44658026674806506, 0.42586033883529295, 0.31490480329261394, 0.11717291190878698, -0.1317369264367395, -0.4469240867650224, -0.43913590220065535, 0.6547411428955072, 0.6443974847152788, 0.5454353653287328, 0.3912086774649248, 0.11303511814718059, -0.43544959265292016], [-0.28477608779987695, -0.3038926525187591, -0.15436905424273706, -0.2965590388433148, -0.3123804753378816, 0.04010745469884547, -0.1315033817302529, -0.3188606259439991, -0.32256787794632147, 0.2838756271817994, 0.13024039580623703, -0.08352824746079546, -0.3080645595383802, -0.3153604861744611, 0.5197424049479351, 0.41439397178966175, 0.2690861751857496, 0.019725746312354464, -0.32764645448708185, -0.3345453758424056, 0.7112810100485064, 0.7044050431103565, 0.6335567282886347, 0.5025012174883221, 0.2615639803675951, -0.3466575078350558], [-0.23062615654506957, -0.2267855438295341, -0.07723106861109286, -0.24862647994079, -0.23916218422066982, 0.12897666012537734, -0.04361138472550763, -0.24895052792835093, -0.23922296607567095, 0.356941663208003, 0.21619457564235944, -0.013250322770752221, -0.24659381909344558, -0.2524683880045534, 0.5715455101013797, 0.4625334861522913, 0.3522339143534312, 0.12797076529258322, -0.23561469915049305, -0.2805403743982873, 0.7414003793793742, 0.7398244465832478, 0.671805175157379, 0.5545266614841735, 0.32203078746322356, -0.25119115114771373], [-0.5405198514740079, -0.5460479790295234, -0.4531270396424133, -0.5349676507854069, -0.5262203899805893, -0.30500447017897436, -0.4171443434257166, -0.5232917647077113, -0.5373871106490451, -0.09262447897685766, -0.24209337747680637, -0.3845688184330368, -0.5453455438257644, -0.5403405797497458, 0.27014690185656154, 0.13487885361841312, -0.07989303830762007, -0.3064132347721892, -0.5284632253757013, -0.5521077323020172, 0.5576605069203773, 0.5676676565683536, 0.4680389414521532, 0.2741258930390068, -0.05030792254215244, -0.5044233343476245], [-0.3142374337681476, -0.32223126009183733, -0.16760974869664158, -0.31791008672784093, -0.32100984497268514, 0.021705991738959263, -0.14674466524478702, -0.3082431568465652, -0.3066131845861976, 0.27600710382851756, 0.09601880872335955, -0.12237562077804054, -0.3192344060017086, -0.3259403844321044, 0.530625400769764, 0.4024567625679446, 0.2535901021868362, -0.029155774748947483, -0.3711462758965579, -0.3672270145351969, 0.7020255953828702, 0.7154656728621078, 0.6393812849314986, 0.5088266027093216, 0.24466006342077018, -0.33804648080740773], [-0.16867106659696718, -0.20392834301791174, -0.021765057504704255, -0.1679800334730791, -0.17782747675329436, 0.19066297828449819, 0.031767194041097735, -0.20722534259700753, -0.18834380832181574, 0.3773187490240285, 0.27112356378947566, 0.07046344773567392, -0.20371802996270838, -0.19148568032043978, 0.5933789372758271, 0.503102401058124, 0.3733224389399088, 0.19409316049585623, -0.21666493212312632, -0.22669541685771222, 0.7532505432786499, 0.7557061994551849, 0.6832755684249951, 0.5743272591426812, 0.3485779131691503, -0.22595750736418796], [-0.11159796642083486, -0.09317444017067267, 0.08307614810781627, -0.08141330171615646, -0.12211850259459212, 0.25423907584767713, 0.09503783744353467, -0.11197235302591468, -0.08801434811886111, 0.4531909412994797, 0.32654524808916374, 0.1739088791049369, -0.16274848275831344, -0.13967760430223208, 0.6202523911430127, 0.5561161810698995, 0.44220052608731986, 0.2511469509521319, -0.09888130816578611, -0.12971761371116208, 0.7767763025925618, 0.7751380450303279, 0.7194812265439992, 0.6237516348194874, 0.4248786806207172, -0.11598670147353374], [-0.030281549592014012, -0.03623605221785409, 0.14462756719337433, -0.04024571605835725, 0.004647940260595007, 0.3445246206996512, 0.16314037082415267, -0.07133347341410257, -0.0009273158153135785, 0.4822161192279816, 0.37320714640988606, 0.21765976191419079, -0.01487751460923481, -0.07072707918700971, 0.6671008703395802, 0.5884415971381808, 0.47241733957875387, 0.283481292180339, -0.030951893934863306, -0.05191679749280731, 0.7888343154886258, 0.7956914355520673, 0.735582421303203, 0.6572441853493891, 0.4698280965446143, -0.056351986847641755], [-0.4324903282572725, -0.43084400760426145, -0.3533870010024578, -0.4451291648679838, -0.43101443527974326, -0.1634997188949996, -0.32458971958016636, -0.44996675863208924, -0.4475318980943809, 0.08511988930514935, -0.10214056082914003, -0.27409368200648676, -0.4337117735164844, -0.42759924782426384, 0.38980360750219134, 0.26774989002198335, 0.08519482525228539, -0.16415127613582037, -0.4660041545494428, -0.4612649242050798, 0.6347790927290551, 0.6399337625007766, 0.5527017924318461, 0.3957320957312752, 0.10616904889324719, -0.4580759349292918], [-0.19457944383623238, -0.18336249445947694, -0.04849219790905612, -0.19287870703337862, -0.2159909841756923, 0.17097344739512166, -0.01412708336340307, -0.18403950387923074, -0.18431042570004508, 0.376847285128766, 0.28763386962101223, 0.03934441457764082, -0.19826184912233324, -0.19972564350193667, 0.5929941718837699, 0.5069429300547594, 0.3632085921518381, 0.11403267262019678, -0.19936601395884213, -0.2136287749701341, 0.751658368281844, 0.7535356002229237, 0.6808705882940623, 0.5946275901632794, 0.37166692164250426, -0.21803415943309046], [-0.021217683545843302, -0.03241199312692988, 0.1721150489618165, -0.010939055672979814, -0.03277473329341518, 0.3025453361728999, 0.1935693800045868, -0.05651707664449934, -0.009638380744136144, 0.5003324797194655, 0.3776791677555421, 0.21623736751050326, -0.04215352036046416, -0.04265539622021919, 0.6558507867153521, 0.5689333702811588, 0.4897810545056287, 0.2983504731068223, -0.07303347861486043, -0.07912632648217732, 0.7960867651427836, 0.794308562580568, 0.74803690432994, 0.6446809915037093, 0.4566959182243242, -0.08785762823174881], [0.03909208302194789, 0.09256716599186954, 0.2162671082358124, 0.05371703740478129, 0.057978529808503376, 0.3881927072422001, 0.23286957386431018, 0.04266140912196614, 0.04425757244098261, 0.5281553387892679, 0.42382282724535625, 0.2829360130506764, 0.07389227783412644, -0.003995021134230448, 0.6881596317454219, 0.6194901382587756, 0.5292538274568221, 0.3625084720108591, 0.01830245145783782, 0.046889848541565526, 0.81171043743647, 0.8176068629490063, 0.7711486536376467, 0.6802460322146309, 0.5157671423918573, 0.043798575210111546], [0.1467801835668311, 0.1315124541723912, 0.26769420313939385, 0.1340755398935326, 0.12518184019662593, 0.44037410520629483, 0.32331141597579627, 0.1387878066049445, 0.11160317106179629, 0.5658705735904532, 0.48568605658535535, 0.34524601819178347, 0.1204383082569187, 0.10675778276236814, 0.7013561941677836, 0.6517069824650534, 0.5628247642642062, 0.4261560612932057, 0.12584758956395348, 0.13126717743401672, 0.8262261605842376, 0.8236842857482392, 0.782618762093136, 0.7148097987993305, 0.5729567485291216, 0.12803604630007315], [0.05875631857119579, 0.09630773037423446, 0.24214801402662556, 0.0983823333266648, 0.07372667494108665, 0.39849605436470825, 0.2666412022074104, 0.06380373736658973, 0.057720281849914845, 0.5421436458217732, 0.44289209523602424, 0.27783642365743244, 0.040436365313955114, 0.09034044140228714, 0.6962574046000662, 0.6345702124057371, 0.5422197228329096, 0.36923292716110856, 0.037856119520505604, 0.018134449047538137, 0.8181499411612934, 0.8138091738537506, 0.7591984858402383, 0.6847510881539691, 0.5286129072976662, 0.011110692380618126], [0.2163843273348517, 0.2364971295979027, 0.3712641509957746, 0.2315151239910167, 0.21497230652731536, 0.5036815645162644, 0.38865868205517895, 0.21469510122437696, 0.2419042343388188, 0.6251188322041874, 0.5313659004045352, 0.43026794004984814, 0.2042660817726336, 0.1580761338521015, 0.7420950976392608, 0.6847751563502458, 0.6023055469579985, 0.4682269893337144, 0.21518633425372347, 0.21869484048580923, 0.8449707173064696, 0.8452610423639313, 0.8086690657731967, 0.7505610781817111, 0.5870073799642818, 0.13940949626917531], [0.31823508684723034, 0.29614686261421874, 0.4250392073668542, 0.31792680438533616, 0.2688617218244099, 0.5427601863769196, 0.43988488848722074, 0.2896286743498071, 0.29942246341045264, 0.6560143787040993, 0.577854104552933, 0.4663934760989755, 0.26909508077050426, 0.2738873139768305, 0.7642124552839873, 0.7186613230236875, 0.6385464351072143, 0.5319437958513382, 0.2750531118650794, 0.29910957044039954, 0.860797015416128, 0.8618351385272992, 0.8209878154727452, 0.7688163002782022, 0.634909807487706, 0.27947478168267836], [0.3607187695244938, 0.3584999403299395, 0.4626795033867286, 0.35582493414582506, 0.3636263462854854, 0.5643166096264487, 0.5087764412147006, 0.3728146689882985, 0.32972589774329286, 0.6843883933251198, 0.620334184395066, 0.5100633199073483, 0.35045720649812917, 0.35208251784149314, 0.7885314770066654, 0.7300682354609476, 0.6800142762715359, 0.5656241900982457, 0.346225861305558, 0.3588764639167582, 0.8704274568697713, 0.8731351989739308, 0.8377983133896523, 0.781541669353598, 0.6821256198933234, 0.3359769701725617], [0.12320479294972353, 0.1419332328374212, 0.2730751078989716, 0.13189069480065307, 0.13049969433502073, 0.4098913634900275, 0.32366368639182486, 0.14342047087647314, 0.11112444504462271, 0.5663005405023167, 0.512774596176697, 0.32398598951703184, 0.12092085858826618, 0.09394928119667034, 0.7064544379487732, 0.6451863663880251, 0.5463137771977642, 0.4138824519303895, 0.10497422797898692, 0.08077587962019817, 0.830504385110473, 0.820943630213017, 0.7788332171233139, 0.6973070714827068, 0.5250323557651857, 0.09826824804556333], [0.1840417735330736, 0.20815675530600974, 0.3354231694768425, 0.18876069597703327, 0.1926705879763081, 0.4702155953443412, 0.3596286946814127, 0.20194253081429814, 0.1982772109439327, 0.6061971882181364, 0.5284813340287178, 0.4070540674573284, 0.16212484612119404, 0.2063509556498585, 0.7323049777548667, 0.6832587544126638, 0.5809545203537961, 0.46678931717904204, 0.20905697285328895, 0.1816959546297151, 0.8391550639917553, 0.8444411693337436, 0.8051412831031554, 0.7252798461168752, 0.6024094847471908, 0.18429644150154112], [0.262489722418197, 0.2813077090510743, 0.3782057381421762, 0.2668389880778116, 0.25916949741955275, 0.5168266937644954, 0.42384724393005646, 0.2792251060277148, 0.274131791573087, 0.643676517103688, 0.5560582996550262, 0.44500729923563, 0.3144198027448627, 0.25413902304230684, 0.760515376710668, 0.7041736854145098, 0.6299069577673788, 0.5191037141807925, 0.2571654637169042, 0.2698814495564228, 0.8474808102514387, 0.8567366496373554, 0.8200402509211336, 0.7571875913735245, 0.6456894586471007, 0.2479871822124864], [0.5285700115570009, 0.5435267914649783, 0.6151676865530229, 0.5076906938499048, 0.5254729126501761, 0.6937604938062292, 0.62317938344043, 0.5247417504632308, 0.5288712468909381, 0.7661446553109268, 0.7158137047700355, 0.6429392020881904, 0.5332143342675035, 0.5144900929473766, 0.8417172716454784, 0.8102916717023285, 0.771981091491621, 0.6902433666216796, 0.5325676972065361, 0.5157139294497518, 0.9026864630503911, 0.903150592963404, 0.882835722408268, 0.8454722598489494, 0.7667360812927808, 0.516114719418151]]

In [291]:
l = ["[1, 2, -]", "[2, 2, -]", "[3, 2, -]", "[4, 2, -]", "[1, 3, -]", "[2, 3, -]", "[3, 3, -]", "[4, 3, -]", "[6, 3, -]", "[1, 4, -]", "[2, 4, -]", "[3, 4, -]", "[4, 4, -]", "[6, 4, -]", "[8, 4, -]", "[2, 5, -]", "[3, 5, -]", "[4, 5, -]", "[6, 5, -]", "[8, 5, -]", "[10, 5, -]", "[3, 6, -]", "[4, 6, -]", "[6, 6, -]", "[8, 6, -]", "[10, 6, -]", "[12, 6, -]", "[16, 6, -]", "[6, 8, -]", "[8, 8, -]", "[10, 8, -]", "[12, 8, -]", "[16, 8, -]", "[1, 2, 6]", "[2, 2, 6]", "[3, 2, 6]", "[4, 2, 6]", "[1, 3, 6]", "[2, 3, 6]", "[3, 3, 6]", "[4, 3, 6]", "[6, 3, 6]", "[1, 4, 6]", "[2, 4, 6]", "[3, 4, 6]", "[4, 4, 6]", "[6, 4, 6]", "[2, 5, 6]", "[3, 5, 6]", "[4, 5, 6]", "[6, 5, 6]", "[3, 6, 6]", "[4, 6, 6]", "[6, 6, 6]", "[6, 8, 6]", "[1, 2, 5]", "[2, 2, 5]", "[3, 2, 5]", "[4, 2, 5]", "[1, 3, 5]", "[2, 3, 5]", "[3, 3, 5]", "[4, 3, 5]", "[6, 3, 5]", "[1, 4, 5]", "[2, 4, 5]", "[3, 4, 5]", "[4, 4, 5]", "[6, 4, 5]", "[2, 5, 5]", "[3, 5, 5]", "[4, 5, 5]", "[6, 5, 5]", "[3, 6, 5]", "[4, 6, 5]", "[6, 6, 5]", "[6, 8, 5]"]


In [292]:
l.reverse()

In [293]:
l

['[6, 8, 5]',
 '[6, 6, 5]',
 '[4, 6, 5]',
 '[3, 6, 5]',
 '[6, 5, 5]',
 '[4, 5, 5]',
 '[3, 5, 5]',
 '[2, 5, 5]',
 '[6, 4, 5]',
 '[4, 4, 5]',
 '[3, 4, 5]',
 '[2, 4, 5]',
 '[1, 4, 5]',
 '[6, 3, 5]',
 '[4, 3, 5]',
 '[3, 3, 5]',
 '[2, 3, 5]',
 '[1, 3, 5]',
 '[4, 2, 5]',
 '[3, 2, 5]',
 '[2, 2, 5]',
 '[1, 2, 5]',
 '[6, 8, 6]',
 '[6, 6, 6]',
 '[4, 6, 6]',
 '[3, 6, 6]',
 '[6, 5, 6]',
 '[4, 5, 6]',
 '[3, 5, 6]',
 '[2, 5, 6]',
 '[6, 4, 6]',
 '[4, 4, 6]',
 '[3, 4, 6]',
 '[2, 4, 6]',
 '[1, 4, 6]',
 '[6, 3, 6]',
 '[4, 3, 6]',
 '[3, 3, 6]',
 '[2, 3, 6]',
 '[1, 3, 6]',
 '[4, 2, 6]',
 '[3, 2, 6]',
 '[2, 2, 6]',
 '[1, 2, 6]',
 '[16, 8, -]',
 '[12, 8, -]',
 '[10, 8, -]',
 '[8, 8, -]',
 '[6, 8, -]',
 '[16, 6, -]',
 '[12, 6, -]',
 '[10, 6, -]',
 '[8, 6, -]',
 '[6, 6, -]',
 '[4, 6, -]',
 '[3, 6, -]',
 '[10, 5, -]',
 '[8, 5, -]',
 '[6, 5, -]',
 '[4, 5, -]',
 '[3, 5, -]',
 '[2, 5, -]',
 '[8, 4, -]',
 '[6, 4, -]',
 '[4, 4, -]',
 '[3, 4, -]',
 '[2, 4, -]',
 '[1, 4, -]',
 '[6, 3, -]',
 '[4, 3, -]',
 '[3, 3, -]',